### Các tầng tùy chỉnh
- Một trong những yếu tố dẫn đến thành công của học sâu là sự đa dạng của các tầng. Những tầng này có thể được sắp xếp theo nhiều cách sáng tạo để thiết kế nên những kiến trúc phù hợp với nhiều tác vụ khác nhau.
- Phần này sẽ hướng dẫn ta xây dựng một tầng tùy chỉnh

#### 1. Các tầng không có tham số
- Để bắt đầu, ta tạo một tầng tùy chỉnh (một khối) không chứa bất kỳ tham số nào. Lớp __CenteredLayer__ chỉ đơn thuần trừ đi giá trị trung bình từ đầu vào của nó. 
- Để xây dựng ta chỉ cần kế thừa từ lớp Block và lập trình phương thức forward.

In [7]:
from mxnet import gluon, np, npx
from mxnet.gluon import nn
npx.set_np()

# **kwargs giúp CenteredLayer có thể kế thừa tất cả các tham số từ nn.Block một cách linh hoạt, 
# Giúp lớp mở rộng mà không cần định nghĩa lại tất cả các tham số của lớp cha.
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

In [8]:
layer = CenteredLayer()
x = np.array(range(0, 5))
layer(x)

array([-2., -1.,  0.,  1.,  2.])

Chúng ta có thể kết hợp tầng này như là một thành phần để xây dựng các mô hình phức tạp hơn.
Để kiểm tra thêm, ta có thể truyền dữ liệu ngẫu nhiên qua mạng và kiểm tra xem chúng thực sự đã có giá trị trung bình về không hay chưa. Do đang làm việc với số thực dấu phẩy động nên ta sẽ thấy một giá trị khác không  rất nhỏ.

In [9]:
net = nn.Sequential()
net.add(nn.Dense(128), CenteredLayer())
net.initialize()

y = net(x)
y.mean()

array(1.1175871e-09)

#### 2. Tầng có tham số
- Ta chuyển sang việc định nghĩa các tầng chứa tham số có thể điều chỉnh được trong quá trình huấn luyện. 
- Để tự động hóa công việc lặp lại, lớp Parameter và từ điển ParameterDict cung cấp một số tính năng quản trị cơ bản gồm truy cập, khởi tạo, chia sẻ, lưu và nạp các tham số mô hình.
- Bằng cách này, cùng với nhiều lợi ích khác, ta không cần phải viết lại các thủ tục tuần tự hóa cho mỗi tầng tùy chỉnh mới.
- Lớp Block chứa biến __params__ với kiểu dữ liệu __ParameterDict__. Từ điển này ánh xạ các xâu ký tự biển thị __tên tham số__  đến các __tham số__  mô hình (Thuộc kiểu __Parameter__). __ParameterDict__ cũng cung cấp hàm __get__ giúp việc tạo tham số mới với tên và chiều cụ thể trở nên dễ dàng.

In [11]:
params = gluon.ParameterDict()
params.get('param2', shape = (2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

Giờ ta đã có tất cả các thành phần cơ bản cần thiết để tự tạo một phiên bản tùy chỉnh của tầng Dense trong Gluon. Chú ý rằng tầng nà yêu cầu hai tham số, trọng số và hệ số điều chỉnh. Trong cách lập trình này, ta sử dụng hàm kích hoạt mặc định là ReLU.
- Trong hàm __ __init__ __, in_units và units biểu thị lần lượt số lượng đầu vào và đầu ra

In [13]:
class MyDense(nn.Block):
    # units: The number of outputs in this layer;
    # in_units: The number of inputs in this layer
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('myweight', shape = (in_units, units))
        self.bias = self.params.get('mybias', shape = (1, units))
    
    def forward(self, x):
        linear = np.dot(x, self.weight.data()) + self.bias.data()
        return npx.relu(linear)

Việc đặt tên cho các tham số cho phép ta truy cập chúng theo tên thông qua tra cứu từ điển sau này. Nhìn chung, ta sẽ muốn đăt cho các biến tên đơn giản biểu thị mục đích rõ ràng của chúng. Tiếp theo, ta sẽ khởi tạo lớp MyDense và truy cập các tham số mô hình. 
Lưu ý rằng tên của khối sẽ được tự động thêm vào trước tên các tham số.

In [14]:
dense = MyDense(units=10, in_units=3)
dense.params

mydense0_ (
  Parameter mydense0_myweight (shape=(3, 10), dtype=<class 'numpy.float32'>)
  Parameter mydense0_mybias (shape=(1, 10), dtype=<class 'numpy.float32'>)
)

In [22]:
dense.initialize(force_reinit=True)
dense(np.random.uniform(low = 5, high = 6, size = (2, 3)))

array([[0.38301522, 0.20578627, 0.        , 0.02557632, 0.21038195,
        0.06228156, 0.40721476, 0.52872777, 0.55179757, 1.0025297 ],
       [0.3586089 , 0.2058548 , 0.        , 0.02261066, 0.2091816 ,
        0.07510042, 0.37659347, 0.46959838, 0.52717686, 0.9419422 ]])

- Các tầng tùy chỉnh có thể được dùng để xây dựng mô hình. Chúng có thể được sử dụng như các tầng được kết nối dày đặc được lập trình sẵn.
- Ngoại lệ duy nhất là việc suy luận kích thước sẽ không được thực hiện tự động.

In [44]:
net = nn.Sequential()
net.add(
    MyDense(10, 3),
    MyDense(3, 10)
)
net.initialize(force_reinit=True)
net(np.random.normal(size = (2, 3)))

array([[0.        , 0.00169223, 0.06004107],
       [0.        , 0.00709664, 0.05866113]])